# Imports

In [126]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import json

from collections import defaultdict

# TO PICKLE FILES

In [180]:
def pickle_it(df,name):
    filepath = '/Users/gandalf/Documents/data/'+name+'.pkl'
    df.to_pickle(filepath)
    pass

# FIRST DATA FILE (USERS)

In [204]:
def get_user_data():
    filename = "/Users/gandalf/Documents/data/data_users.csv"

    user_df = pd.read_csv(filename, parse_dates=['created', 
                                            'updated', 
                                            'available', 
                                            'birthday', 
                                            'lastActive'])

    user_df = user_df.fillna({'about'    : '', 
                    'birthday' : pd.to_datetime('1899-01-01'), 
                    'latitude' : 0, 
                    'longitude': 0})

    user_df['len_about'] = user_df.about.apply(lambda x: len(x))
    user_df['has_about'] = user_df.len_about > 0
    user_df['age'] = 2018-user_df['birthday'].apply(lambda x: x.year)

    col_to_drop = ['id']
    user_df = user_df.drop(col_to_drop, axis=1)


    user_df = user_df[['uid', 'created', 'updated', 'about','has_about', 'len_about', 'available',
           'birthday', 'age', 'collegeId', 'emailVerified', 'foundRoommate', 'gender',
           'groupChat', 'hometownId', 'inRelationship', 'isClean', 'isNight',
           'isStudent', 'lastActive', 'latitude', 'longitude', 'maxCost',
           'minCost', 'numRoommates', 'onboarded', 'petsOk', 'pictureId',
           'roomPostId', 'roomTypeId', 'smokingOk', 'term', 'work']]

    user_df = user_df.drop_duplicates()
    user_df = user_df.set_index('uid')
    return user_df

In [438]:
user_df = get_user_data()
user_df.head(2)

,created,updated,about,has_about,len_about,available,birthday,age,collegeId,emailVerified,...,minCost,numRoommates,onboarded,petsOk,pictureId,roomPostId,roomTypeId,smokingOk,term,work
uid,,,,,,,,,,,,,,,,,,,,,
0a9yOPKFSH,2017-06-19 03:31:08,2018-01-04 21:24:41,,False,0,nan,1992-01-01,26,NaN,1,...,850.0,2.0,1.0,1.0,331.0,375.0,2.0,0.0,12.0,SEG
0owxBbcyZ6,2017-10-12 00:32:29,2018-01-04 03:05:50,,False,0,nan,1974-01-01,44,NaN,1,...,NaN,NaN,1.0,1.0,516.0,89.0,NaN,1.0,NaN,WA State


# SECOND DATA FILE (MESSAGES)

In [315]:
def get_messages():
    filename = "/Users/gandalf/Documents/data/data_messages.json"
    json_data=open(filename).read()
    data = json.loads(json_data)

    column_names = ['message_id',       # Added, unique for each row
                   'conversation_id',   # Added, identifies the conversation, not unique
                   'uid',               # message sender
                    's_uid',             # message receiver
                   'read',              # true/false value
                   'readBy',            # message recipient(s)
                   'text_length',       # Added, length of text
                   'timestamp',         # time
                   'imageURL',          # image url
                   'emailAttempted',    # ?
                   'emailed']           # ?

    mi,ci,ui,si,rd,rb,tl,ts,iu,ea,em = [],[],[],[],[],[],[],[],[],[],[]
    c = 0
    for conversation_id, conversation_data in data["conversations"].items():
        for message_id, message_data in conversation_data.items():
            information = defaultdict(lambda: '', message_data)
            ci.append(conversation_id)
            mi.append(message_id)
            ui.append(information['uid'])
#             if information['uid'] == conversation_id[:10]:
#                 si.append(conversation_id[10:])
#             elif information['uid'] == conversation_id[10:]:
#                 si.append(conversation_id[:10])
#             else: c += 1
            si.append(None)
            rd.append(information['read'])
            rb.append(information['readBy'])
            tl.append(len(information['text'].split()))
            ts.append(information['timestamp'])
            iu.append(information['imageURL'])
            ea.append(information['emailAttempted'])
            em.append(information['emailed'])
    print(c)
    df = pd.DataFrame([mi,ci,ui,si,rd,rb,tl,ts,iu,ea,em]).T
    df.columns=column_names

    df['date'] = df.timestamp.apply(lambda x: pd.to_datetime(x*1000000))

    df = df[['message_id', 'conversation_id', 'uid','s_uid', 'read', 'readBy', 'text_length',
           'timestamp','date', 'imageURL', 'emailAttempted', 'emailed']]

    
        # going to make an assumption here, and fix later if needed
#     for index, row in df.iterrows()
#         if row.first_uid == row.first_ten:
#             row.second_uid = row.last_ten
#         elif row.first_uid == row.last_ten:
#             row.second_uid = row.first_ten
#     else:
#         print("panic at the disco!")
    return df
df = get_messages()
df.head(1)

0


,message_id,conversation_id,uid,s_uid,read,readBy,text_length,timestamp,date,imageURL,emailAttempted,emailed
0,-KdmSRA-1a7niRVp5s83,-KdmSL0izd_44wrniy-W,8Ws3QWmC4B,None,True,"{'1': '6y3bPICtHM', '0': 'OBFaJBVxWC'}",3,1487979004690,2017-02-24 23:30:04.690,,,


In [400]:
def get_conversations():
    filename = "/Users/gandalf/Documents/data/data_messages.json"
    json_data=open(filename).read()
    data = json.loads(json_data)
    
    message_df['const'] = 1
    convo_length = message_df.groupby('conversation_id').const.sum().T.to_dict()

    column_names = ['conv_id',       # user id
                    'response',      # did anyone respond?
                    'first_uid',     # user who sent the first message
                    'second_uid',    # user who responded
                    'first_mid',     # message id of the first message
                    'second_mid',    # message id of the second message
                    'timestamp'
                   ]   

    ci,rs,fu,fm,su,sm,ts = [],[],[],[],[],[],[]
    already_added = set()

    first_message = True
    second_message = False
    c = 0
    for index, row in message_df.iterrows():
        if convo_length[row.conversation_id] == 1:
            temp = row.conversation_id
            ci.append(temp)
            rs.append(False)
            fu.append(row.uid)
            fm.append(row.message_id)
#             if information['uid'] == conversation_id[:10]:
#                 su.append(conversation_id[10:])
#             elif information['uid'] == conversation_id[10:]:
#                 su.append(conversation_id[:10])
#             else: c += 1
            su.append(None)
            sm.append(None)
            ts.append(row.timestamp)

        elif row.conversation_id not in already_added:
            if first_message:
                temp = row.conversation_id
                ci.append(temp)
                ft.append(temp[:10])
                lt.append(temp[10:])
                rs.append(True)
                fu.append(row.uid)
                fm.append(row.message_id)
                first_message = False
                second_message = True
                ts.append(row.timestamp)
            elif second_message:
                su.append(row.uid)
                sm.append(row.message_id)
                already_added.add(row.conversation_id)
                first_message = True
                second_message = False

    df = pd.DataFrame([ci,rs,fu,su,fm,sm,ts]).T
    df.columns=column_names
    
    df['first_ten'] = df.conv_id.apply(lambda x: x[:10])
    df['last_ten'] = df.conv_id.apply(lambda x: x[10:])
    
    df=df[24:]
    
    # going to make an assumption here, and fix later if needed
    c = 0
    for index, row in df.iterrows():
        if row.first_ten == row.first_uid: row.second_uid = row.last_ten
        elif row.last_ten == row.first_uid: row.second_uid = row.first_ten
        else:c += 1

    df = df.drop(['first_ten', 'last_ten'], axis=1)
    
    print("if this number is not zero, you have a problem: {}".format(c))
    
    df = df.sort_values(['timestamp'])
    df=df.reset_index()
    return df

In [198]:
def get_lastmessage():
    filename = "/Users/gandalf/Documents/data/data_messages.json"
    json_data=open(filename).read()
    data = json.loads(json_data)

    column_names = ['user_id',       # user id
                   'first_ten',      # first ten of the 20 digit key
                   'last_ten',       # last ten of the 20 digit key (prob userid)
                   'lastMessageId']  # message id

    ui,ft,lt,lm = [],[],[],[]

    for user_id, user_data in data['users'].items():
        for key, value in user_data['conversations'].items():
            ui.append(user_id)
            ft.append(key[:10])
            lt.append(key[10:])
            lm.append(value['lastMessageId'])

    lastmessage_df = pd.DataFrame([ui,ft,lt,lm]).T
    lastmessage_df.columns=column_names
    
    return lastmessage_df

# READ IN and ECT SPLIT

In [431]:
# read in
message_df = get_messages()
convo_df = get_conversations()
last_df = get_lastmessage()

0
if this number is not zero, you have a problem: 0


In [411]:
# instructions for split
def ect_find_split(df, percent):
    cutoff_index = int(len(df)*percent)-1
    return df.loc[cutoff_index,'timestamp']

def ect_make_split(df, cutoff_timestamp):
    new_df = df[df.timestamp <= cutoff_timestamp]
    return new_df

In [430]:
# do the split!
cutoff = ect_find_split(convo_df, .7)
new_convo_df = ect_make_split(convo_df, cutoff)

2017-10-27 23:26:57.659000
2017-10-27 23:26:57.659000


# COMBINE USER AND MESSAGE FILES

In [435]:
# Get responsiveness table

response_df['const'] = 1

# how many did he send?
messages_sent = response_df.groupby('first_uid').const.sum()
# how many resposnes did he get?
responses_received = response_df[response_df.response == True].groupby('first_uid').const.sum()
# ratio (higher = more attractive roommmate)
attractiveness = responses_received/messages_sent

# how many did he receive?
messages_received = response_df.groupby('second_uid').const.sum()
# how many did he respond to?
responses_sent = response_df[response_df.response == True].groupby('first_uid').const.sum()
# ratio (higher = responds less)
selectivity = 1-responses_sent/messages_received

user_response = pd.concat([messages_sent, responses_received, attractiveness,
                           messages_received, responses_sent, selectivity], axis=1)
user_response.columns=[['messages_sent', 'responses_received', 'attractiveness',
                           'messages_received', 'responses_sent', 'selectivity']]
user_response = user_response.fillna(0)

user_response.head()

,messages_sent,responses_received,attractiveness,messages_received,responses_sent,selectivity
013LzOrVju,21.0,1.0,0.047619,2.0,1.0,0.500000
01iDVeaXl1,4.0,0.0,0.000000,1.0,0.0,0.000000
02ICQW0SJo,7.0,4.0,0.571429,14.0,4.0,0.714286
02Wvx6BcIx,1.0,0.0,0.000000,0.0,0.0,0.000000
02omVxM8Xm,1.0,0.0,0.000000,0.0,0.0,0.000000


In [439]:
# Combine with user table
user_df = user_df.join(user_response)
# user_df = user_df.fillna({'sent' : 0,'responses': 0, 'ratio' :0})
user_df.head()

,created,updated,about,has_about,len_about,available,birthday,age,collegeId,emailVerified,...,roomTypeId,smokingOk,term,work,messages_sent,responses_received,attractiveness,messages_received,responses_sent,selectivity
uid,,,,,,,,,,,,,,,,,,,,,
0a9yOPKFSH,2017-06-19 03:31:08,2018-01-04 21:24:41,,False,0,nan,1992-01-01,26,NaN,1,...,2.0,0.0,12.0,SEG,7.0,2.0,0.285714,7.0,2.0,0.714286
0owxBbcyZ6,2017-10-12 00:32:29,2018-01-04 03:05:50,,False,0,nan,1974-01-01,44,NaN,1,...,NaN,1.0,NaN,WA State,NaN,NaN,NaN,NaN,NaN,NaN
0jWle2dnry,2017-10-14 17:25:41,2017-10-14 17:39:20,,False,0,nan,1977-01-01,41,NaN,1,...,NaN,NaN,NaN,Lakeshore,NaN,NaN,NaN,NaN,NaN,NaN
02omVxM8Xm,2017-09-11 02:33:40,2018-01-04 00:24:19,The perfect place... wya ?,True,26,nan,1996-01-01,22,NaN,1,...,NaN,1.0,NaN,Delhi Palace,1.0,0.0,0.000000,0.0,0.0,0.000000
0EWNOmyQmW,2017-07-29 23:03:41,2018-01-04 22:41:08,Roommate is moving out and looking for someone...,True,377,nan,1986-01-01,32,536.0,1,...,NaN,0.0,NaN,Insightly,NaN,NaN,NaN,NaN,NaN,NaN


# PICKLE EVERYTHING!

https://www.youtube.com/watch?v=yYey8ntlK_E

In [441]:
pickle_it(user_df,'user_df')
pickle_it(message_df,'message_df')
pickle_it(convo_df,'convo_df')
pickle_it(lastmessage_df,'lastmessage_df')

# INVESTIGATE A CONVERSATION BY ID

In [ ]:
convo_id = '20jHWHiCJr1xaCe4PTSe'

for conversation_id, conversation_data in data["conversations"].items():
    if conversation_id == convo_id:
        for message_id, message_data in conversation_data.items():
            information = defaultdict(lambda: '', message_data)
            print(information['text'])
            
message_df[message_df.conversation_id == convo_id]

# INVESTIGATE MESSAGES SENT BY A UID

In [ ]:
goal_uid = '7MuruIIPpJ'

for conversation_id, conversation_data in data["conversations"].items():
    for message_id, message_data in conversation_data.items():
#         print(message_data)
        if information.uid == goal_uid:
            print(information['text'])
            
# message_df[message_df.conversation_id == convo_id]

In [ ]:
goal_uid = '7MuruIIPpJ'

for conversation_id, conversation_data in data["conversations"].items():
    for message_id, message_data in conversation_data.items():
        try:
            if message_data['uid']==goal_uid:
                print(information['text'])
        except:
            pass